In [38]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.neighbors import RadiusNeighborsClassifier
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import warnings


In [39]:
#загрузка данных
df_train = pd.read_csv("../input/novosib/train_dataset_train_.csv")
df_test = pd.read_csv("../input/novosib/test_dataset_test.csv")
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)


In [40]:
#получение новых меток [0, 1, 2, 3, 4, 5]
df_train['Class'] = df_train['Class'].map({0:0, 1:1, 3:2, 4:3, 5:4, 64:5 })


In [41]:
#получение из тренировочного датасета пространственных координат (x, y, z) 
#и целевой переменной принадлежности к классу y
X = df_train.drop(["Class", "id", 'Reflectance'], axis = 1).values# ('Easting', 'Northing','Height')
y = df_train["Class"].values


In [42]:
#получение из тестового датасета пространственных координат (x, y, z) 
#и сохранение id номеров тестового датасета
test_id = df_test['id'].values#('Easting', 'Northing','Height')
X_test = df_test.drop(['id', 'Reflectance'], axis=1).values

In [43]:
#нормализация тренировочных и тестовых данных
ss= StandardScaler()
X_scaled = ss.fit_transform(X)
X_t_scaled =ss.fit_transform(X_test)

In [44]:
#создание дополнительного столбца 'neighr' для определение плотности точек тренировочного датасета, 
#с использованием алгоритма RadiusNeighborsClassifier и встроенного метода radius_neighbors 
#определение числа точек, находящихся в шаре радиуса r = 0.034 с центром в этой точке
df_train['neighr'] = 0
neigh = RadiusNeighborsClassifier(radius=0.034)
neigh.fit(X_scaled, y) 
for i in range(1, len(X_scaled) // 200000 + 2):
    print(i)
    df_train.iloc[(i - 1) * 200000:i * 200000, -1] = list(map(len, neigh.radius_neighbors(X_scaled[(i - 1) * 200000:i * 200000], 0.034,  return_distance=False)))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [45]:
#создание дополнительного столбца 'neighr' для определение плотности точек ттестового датасета 
#с использованием алгоритма RadiusNeighborsClassifier и встроенного метода radius_neighbors
#определение числа точек, находящихся в шаре радиуса r = 0.034 с центром в этой точке
df_test['neighr'] = 0
for i in range(1, len(X_t_scaled) // 200000 + 2):
    print(i)
    df_test.iloc[(i - 1) * 200000:i * 200000, -1] = list(map(len, neigh.radius_neighbors(X_t_scaled[(i - 1) * 200000:i * 200000], 0.034,  return_distance=False)))

1
2
3
4
5
6
7
8
9
10


In [46]:
#получение тренировочного датасета со столбцами ('Easting', 'Northing','Height','Reflectance', 'neighr') и целевой переменной y
X = df_train.drop(["Class", "id"], axis = 1).values
y = df_train["Class"].values

In [47]:
#получение тестового датасета со столбцами ('Easting', 'Northing','Height','Reflectance', 'neighr')
test_id = df_test['id'].values
X_test = df_test.drop(['id'], axis=1).values

In [48]:
#инициализация классификатора XGBoost
estimator = xgb.XGBClassifier(random_state=7575, max_depth=7, n_estimators=100, learning_rate=0.3, tree_method='gpu_hist')            

In [63]:
#настройка параметров модели и предобработка тренировочного датасета
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)#применение стратифицированной кросс валидации
with warnings.catch_warnings():
    warnings.simplefilter("ignore")        
    oversample = SMOTE('not majority', k_neighbors = 4)
    X_new, y_new = oversample.fit_resample(X, y)
    #применение оверсэмплинга для тренировочного датасета для выравневания 
    #количества значений в каждом классе по максимальному классу
    for train_index, val_index in kfold.split(X_new, y_new):
                X_train, X_val = X_new[train_index], X_new[val_index]
                y_train, y_val = y_new[train_index], y_new[val_index]
                estimator.fit(X_train, y_train)
                #получение значений метрики recall для каждого из классов
                display(estimator, recall_score(y_val, estimator.predict(X_val), average=None))
                #получение усредненной метрики recall по всем 6 классам
                display(sum(recall_score(y_val, estimators[i].predict(X_val), average=None) / 6))
                break

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=7575, reg_alpha=0, ...)

array([0.9641763 , 0.99827848, 0.95691043, 0.99658259, 0.99635662,
       0.98557092])

0.9829792236869666

In [64]:
#Обучение модели
estimator.fit(X_new, y_new)
y_pred = estimator.predict(X_test)

In [65]:
# возврат к старым меткам и запись в файл
sub = pd.DataFrame({'id':list(test_id), 'Class':y_pred})
sub['Class'] = sub['Class'].map({0:0, 1:1, 2:3, 3:4, 4:5, 5:64 })
sub.to_csv('sub.csv', index=False)